In [19]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pretty_midi
import pandas as pd
import os

In [20]:
def Li_of_Dict_MIDI_MFCCs (mp3_file_path, midi_file_path, n_mfcc=13, hop_length=512):
    
    # load MP3
    y, sr = librosa.load(mp3_file_path, sr=None) # with origional sample rate
    
    # Compute MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
    
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)
    # Extract note information (start time, end time, pitch, velocity)
    notes = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            notes.append({
                'start_time': note.start,
                'end_time': note.end,
                'pitch': note.pitch,
                'velocity': note.velocity
            })
    # DF for notes information
    midi_df = pd.DataFrame(notes)
    
    # creates paired data container as a list of dictionaries for every audio file
    notes_mfccs = []

    # Iterate through MIDI notes
    for _, row in midi_df.iterrows():
        start_time = row['start_time']
        end_time = row['end_time']
        pitch = row['pitch']
        velocity = row['velocity']

        # Find MFCC frames within the MIDI note time range
        mfccs_within_range = mfccs[:, int(start_time * sr / hop_length):int(end_time * sr / hop_length)]

        # Append aligned data as a dictionary to the notes_mfccs list
        notes_mfccs.append({'start_time': start_time,'end_time': end_time,'pitch': pitch,'velocity': velocity,'mfccs': mfccs_within_range})

    # Now 'notes_mfccs' contains aligned data
    print('out putted: ',mp3_file_path)
    return notes_mfccs

In [21]:
def get_file_path_audio_and_midi (audio_dir = "piano_mp3", midi_dir = "piano_midi"):
    
    # Get paths for files in the audio folder
    audio_paths = [os.path.join(audio_dir, filename) for filename in os.listdir(audio_dir) if os.path.isfile(os.path.join(audio_dir, filename))]
    audio_paths.sort()
    # Get paths for files in the midi folder
    midi_paths = [os.path.join(midi_dir, filename) for filename in os.listdir(midi_dir) if os.path.isfile(os.path.join(midi_dir, filename))]
    midi_paths.sort()
    
    return audio_paths, midi_paths

In [28]:
# This chunk of code gets the big data as a list of lists(songs) of dictionaries(notes)
mp3, midi = get_file_path_audio_and_midi()

data = []
for mp3, midi in zip(mp3, midi):
    try:
        music_data = Li_of_Dict_MIDI_MFCCs(mp3, midi)
        data.append(music_data)
    except OSError:
        print("Error: failed to gather data for: ", mp3)


out putted:  piano_mp3\alb_esp1.mp3
out putted:  piano_mp3\alb_esp2.mp3
out putted:  piano_mp3\alb_esp3.mp3
out putted:  piano_mp3\alb_esp4.mp3
out putted:  piano_mp3\alb_esp5.mp3
out putted:  piano_mp3\alb_esp6.mp3
out putted:  piano_mp3\alb_se1.mp3
out putted:  piano_mp3\alb_se2.mp3
out putted:  piano_mp3\alb_se3.mp3
out putted:  piano_mp3\alb_se4.mp3
out putted:  piano_mp3\alb_se5.mp3
out putted:  piano_mp3\alb_se6.mp3
out putted:  piano_mp3\alb_se7.mp3
out putted:  piano_mp3\alb_se8.mp3
out putted:  piano_mp3\appass_1.mp3
out putted:  piano_mp3\appass_2.mp3
out putted:  piano_mp3\appass_3.mp3
out putted:  piano_mp3\bach_846.mp3
out putted:  piano_mp3\bach_847.mp3
out putted:  piano_mp3\bach_850.mp3
out putted:  piano_mp3\beethoven_hammerklavier_1.mp3
out putted:  piano_mp3\beethoven_hammerklavier_2.mp3
out putted:  piano_mp3\beethoven_hammerklavier_3.mp3
out putted:  piano_mp3\beethoven_hammerklavier_4.mp3
out putted:  piano_mp3\beethoven_les_adieux_1.mp3
out putted:  piano_mp3\bee

out putted:  piano_mp3\mendel_op30_1.mp3
out putted:  piano_mp3\mendel_op30_2.mp3
out putted:  piano_mp3\mendel_op30_3.mp3
out putted:  piano_mp3\mendel_op30_4.mp3
out putted:  piano_mp3\mendel_op30_5.mp3
out putted:  piano_mp3\mendel_op53_5.mp3
out putted:  piano_mp3\mendel_op62_3.mp3
out putted:  piano_mp3\mendel_op62_4.mp3
out putted:  piano_mp3\mendel_op62_5.mp3
out putted:  piano_mp3\mond_1.mp3
out putted:  piano_mp3\mond_2.mp3
out putted:  piano_mp3\mond_3.mp3
out putted:  piano_mp3\mos_op36_6.mp3
out putted:  piano_mp3\muss_1.mp3
out putted:  piano_mp3\muss_2.mp3
out putted:  piano_mp3\muss_3.mp3
out putted:  piano_mp3\muss_4.mp3
out putted:  piano_mp3\muss_5.mp3
out putted:  piano_mp3\muss_6.mp3
out putted:  piano_mp3\muss_7.mp3
out putted:  piano_mp3\muss_8.mp3
out putted:  piano_mp3\mz_311_1.mp3
out putted:  piano_mp3\mz_311_2.mp3
out putted:  piano_mp3\mz_311_3.mp3
out putted:  piano_mp3\mz_330_1.mp3
out putted:  piano_mp3\mz_330_2.mp3
out putted:  piano_mp3\mz_330_3.mp3
out

In [31]:
# save my data to a file
import pickle

# save to a file
with open("piano_data.pkl", "wb") as f:
    pickle.dump(data, f)